In [105]:
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp

In [106]:
#Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

In [107]:
filename = 'wheat-seeds.csv'
dataset = load_csv(filename)
#Print 5 first rows
dataset[:5]

[['15.26', '14.84', '0.871', '5.763', '3.312', '2.221', '5.22', '1'],
 ['14.88', '14.57', '0.8811', '5.554', '3.333', '1.018', '4.956', '1'],
 ['14.29', '14.09', '0.905', '5.291', '3.337', '2.699', '4.825', '1'],
 ['13.84', '13.94', '0.8955', '5.324', '3.379', '2.259', '4.805', '1'],
 ['16.14', '14.99', '0.9034', '5.658', '3.562', '1.355', '5.175', '1']]

In [108]:
len(dataset)

210

In [109]:
len(dataset[0])

8

In [110]:
#Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
#Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

In [111]:
#Convert the features in dataset from string to float, convert class to integer
for i in range(len(dataset[0])-1):
    str_column_to_float(dataset, i)
str_column_to_int(dataset, len(dataset[0])-1)
dataset[:5]

[[15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22, 0],
 [14.88, 14.57, 0.8811, 5.554, 3.333, 1.018, 4.956, 0],
 [14.29, 14.09, 0.905, 5.291, 3.337, 2.699, 4.825, 0],
 [13.84, 13.94, 0.8955, 5.324, 3.379, 2.259, 4.805, 0],
 [16.14, 14.99, 0.9034, 5.658, 3.562, 1.355, 5.175, 0]]

In [112]:
#Find the minimum and maximum values for each column in the dataset
def dataset_minimax(dataset):
    minmax = list()
    stats = [[min(column),max(column)] for column in zip(*dataset)]
    return stats
#Rescale the values in the dataset to the range 0-1
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)-1):
            row[i] = (row[i]-minmax[i][0])/(minmax[i][1]-minmax[i][0])

In [113]:
#Normalize features in the dataset
minmax = dataset_minimax(dataset)
normalize_dataset(dataset,minmax)
#Show first five rows of dataset
dataset[:5]

[[0.4409820585457979,
  0.5020661157024793,
  0.570780399274047,
  0.48648648648648646,
  0.48610121168923714,
  0.18930164220052273,
  0.3451501723289019,
  0],
 [0.40509915014164316,
  0.44628099173553726,
  0.6624319419237747,
  0.3688063063063065,
  0.5010691375623664,
  0.03288301759221938,
  0.21516494337764666,
  0],
 [0.3493862134088762,
  0.3471074380165289,
  0.8793103448275864,
  0.22072072072072094,
  0.50392017106201,
  0.25145301590191005,
  0.15066469719350079,
  0],
 [0.3068932955618508,
  0.31611570247933873,
  0.7931034482758617,
  0.23930180180180172,
  0.5338560228082679,
  0.19424254638598865,
  0.14081733136386,
  0],
 [0.5240793201133145,
  0.5330578512396694,
  0.8647912885662429,
  0.4273648648648651,
  0.6642908054169634,
  0.076701036289641,
  0.32299359921221066,
  0]]

In [114]:
#Split a dataset into k-folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = len(dataset)//n_folds
    for i in range(n_folds):
        fold = list()
        while len(fold)<fold_size:
            index = randrange(len(dataset_copy)) #Get random data
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [115]:
#Calculate accuracy
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i]==predicted[i]:
            correct+=1
    return correct/float(len(actual))*100.0

In [116]:
#Evaluate algorithm using cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset,n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set,[])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
        return scores

In [117]:
#Calculate the activation of one neuron given an input
def activate(weights, inputs):
    activation = weights[-1] #Bias
    for i in range(len(weights)-1):
        activation += weights[i]*inputs[i]
    return activation

In [118]:
#Neuron Transfer
#Transform the values from the neuron to output
#For example sigmoid function
def sigmoid(activation):
    return 1.0/(1.0+exp(-activation))

In [119]:
#Forward propagate input to a network output
#Input: network, row(data)
def forward_propagate(network, row):
    inputs = row #Data as the inputs to the first hidden layer
    for layer in network:
        new_inputs = []
        #Iterate through every layer
        for neuron in layer:
            activation = activate(neuron['weights'], inputs) #Calculates activation at each neuron in the layer
            neuron['output'] = sigmoid(activation) #Calculates output from the activation at each neuron
            new_inputs.append(neuron['output']) #The outputs is used as the inputs to the next layer
        inputs = new_inputs
    return inputs

In [120]:
#Transfer Derivative
#Calculate slope from an output value of a neuron
def transfer_derivative_sigmoid(output):
    return output*(1.0-output)

In [121]:
#Error Backpropagation
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i!= len(network)-1: #For the hidden layer
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i+1]:
                    error += (neuron['weights'][j]*neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)): #Iterate through neurons at output layer
                neuron = layer[j]
                errors.append(expected[j]-neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j]*transfer_derivative_sigmoid(neuron['output'])

In [122]:
#Update Weights
def update_weights(network, row, learning_rate):
    #Iterate through layers
    for i in range(len(network)): 
        inputs = row[:-1]
        #Except the first hidden layer
        if i!=0: 
            inputs = [neuron['output'] for neuron in network[i-1]] #The inputs is the output from the previous layer
        for neuron in network[i]: #Iterate through neurons
            for j in range(len(inputs)):
                   neuron['weights'][j] += learning_rate*neuron['delta']*inputs[j]
            neuron['weights'][-1] += learning_rate*neuron['delta'] #For bias

In [123]:
#Train network
def train_network(network, train, learning_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, learning_rate)
        print('>Epoch=%d, learning_rate=%.3f, error=%.3f' %(epoch, learning_rate, sum_error))

In [124]:
#Has three parameters: the number of inputs (number of features), the number of neurons to have in the hidden layer,
#and the number of outputs
def initialize_network(n_inputs,n_hidden, n_outputs):
    network = list()
    #At each neuron in the hidden layer, it has n_inputs+1 weights that is initially randomized between 0 to 1
    hidden_layer = [{'weights': [random() for i in range(n_inputs+1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    #At each neuron in the hidden layer, it has n_hidden+1 weights (correspond to each neuron in the hidden layer) 
    #that is initially randomized between 0 to 1
    output_layer = [{'weights': [random() for i in range(n_hidden+1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

In [125]:
#Function to calculate prediction
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

In [126]:
#Backpropagation Algorithm with Stochastic Gradient Descent
def back_propagation(train, test, learning_rate, n_epoch, n_hidden):
    n_inputs = len(train[0])-1
    n_outputs = len(set([row[-1] for row in train]))
    network = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, learning_rate, n_epoch, n_outputs)
    predictions = list()
    for row in test:
        prediction = predict(network, row)
        predictions.append(prediction)
    return predictions

In [128]:
seed(1)
#Evaluate algorithm
n_folds = 5
learning_rate = 0.3
n_epoch = 500
n_hidden = 10
scores = evaluate_algorithm(dataset, back_propagation, n_folds, learning_rate, n_epoch, n_hidden)
print('Scores: %s' %scores)
print('Mean Accuracy: %.3f%%' %(sum(scores)/float(len(scores))))

>Epoch=0, learning_rate=0.300, error=230.469
>Epoch=1, learning_rate=0.300, error=155.588
>Epoch=2, learning_rate=0.300, error=109.636
>Epoch=3, learning_rate=0.300, error=99.606
>Epoch=4, learning_rate=0.300, error=85.425
>Epoch=5, learning_rate=0.300, error=72.398
>Epoch=6, learning_rate=0.300, error=63.877
>Epoch=7, learning_rate=0.300, error=58.233
>Epoch=8, learning_rate=0.300, error=53.712
>Epoch=9, learning_rate=0.300, error=49.494
>Epoch=10, learning_rate=0.300, error=45.356
>Epoch=11, learning_rate=0.300, error=41.399
>Epoch=12, learning_rate=0.300, error=37.826
>Epoch=13, learning_rate=0.300, error=34.775
>Epoch=14, learning_rate=0.300, error=32.262
>Epoch=15, learning_rate=0.300, error=30.225
>Epoch=16, learning_rate=0.300, error=28.574
>Epoch=17, learning_rate=0.300, error=27.223
>Epoch=18, learning_rate=0.300, error=26.103
>Epoch=19, learning_rate=0.300, error=25.161
>Epoch=20, learning_rate=0.300, error=24.358
>Epoch=21, learning_rate=0.300, error=23.664
>Epoch=22, learni

>Epoch=185, learning_rate=0.300, error=11.648
>Epoch=186, learning_rate=0.300, error=11.619
>Epoch=187, learning_rate=0.300, error=11.590
>Epoch=188, learning_rate=0.300, error=11.561
>Epoch=189, learning_rate=0.300, error=11.532
>Epoch=190, learning_rate=0.300, error=11.503
>Epoch=191, learning_rate=0.300, error=11.474
>Epoch=192, learning_rate=0.300, error=11.446
>Epoch=193, learning_rate=0.300, error=11.417
>Epoch=194, learning_rate=0.300, error=11.388
>Epoch=195, learning_rate=0.300, error=11.360
>Epoch=196, learning_rate=0.300, error=11.331
>Epoch=197, learning_rate=0.300, error=11.303
>Epoch=198, learning_rate=0.300, error=11.275
>Epoch=199, learning_rate=0.300, error=11.247
>Epoch=200, learning_rate=0.300, error=11.219
>Epoch=201, learning_rate=0.300, error=11.191
>Epoch=202, learning_rate=0.300, error=11.164
>Epoch=203, learning_rate=0.300, error=11.136
>Epoch=204, learning_rate=0.300, error=11.109
>Epoch=205, learning_rate=0.300, error=11.082
>Epoch=206, learning_rate=0.300, e

>Epoch=366, learning_rate=0.300, error=7.904
>Epoch=367, learning_rate=0.300, error=7.886
>Epoch=368, learning_rate=0.300, error=7.868
>Epoch=369, learning_rate=0.300, error=7.850
>Epoch=370, learning_rate=0.300, error=7.832
>Epoch=371, learning_rate=0.300, error=7.814
>Epoch=372, learning_rate=0.300, error=7.796
>Epoch=373, learning_rate=0.300, error=7.779
>Epoch=374, learning_rate=0.300, error=7.761
>Epoch=375, learning_rate=0.300, error=7.743
>Epoch=376, learning_rate=0.300, error=7.725
>Epoch=377, learning_rate=0.300, error=7.707
>Epoch=378, learning_rate=0.300, error=7.690
>Epoch=379, learning_rate=0.300, error=7.672
>Epoch=380, learning_rate=0.300, error=7.654
>Epoch=381, learning_rate=0.300, error=7.636
>Epoch=382, learning_rate=0.300, error=7.619
>Epoch=383, learning_rate=0.300, error=7.601
>Epoch=384, learning_rate=0.300, error=7.583
>Epoch=385, learning_rate=0.300, error=7.566
>Epoch=386, learning_rate=0.300, error=7.548
>Epoch=387, learning_rate=0.300, error=7.530
>Epoch=388